# Model Coupling Master Logfile

## 2017-06-12

* Taking back up where we left off in September 2016

  - Created a new folder

        ~/SoC_Research/OceanAtmoshpereModelCoupling

      This folder combines the model coupling related material in

        ~/Git/gmcf (copied and cleaned up)

      and
        ~/201608_IOD_Saji (moved)

* In `gmcf`, TODO:
  - Make sure Gordon's changes are integrated but `#ifdef`-ed
  - Looks like I had been working to replace gmcfAPImodel1 by gmcfAPIocean and gmcfAPImodel2 by gmcfAPIatmosphere. But it is quite incomplete: the ocean model should send a temperature profile to the atmosphere as well. And of course we need to do all the proper interpolation
  
* Action plan
	1. Test the skeleton for coupling, ensure it does the right thing and debug (most likely)
	2. Replace the gmcfAPImodel* by the more realistic ocean and atmosphere APIs, test again
	3. Create the time and space interpolation and test
	4. Integrate into the gmcfAPI{ocean,atmoshpere} code
	5. Refactor and build the original ocean and atmosphere models.
	6. Replace the stub generators for temperature and wind profiles with the actual models
	7. Create realistic test runs
	

* Step 1:
	- Problem 1: Didn't build anymore.
	- Fix 1: add `quadmath` to `LIBS` in `SConstruct.OCEAN_ATMOSPHERE`
	
The code seems to work but it is currently a producer-consumer where model 2 does 10 time steps while model 1 does 200.
- Model 1 requests data from model 2 (var_name_1,var_name_1_prev,var_name_2,var_name_2_prev) before it acts on the data
	and does nothing afterwards
- Model 2 sends var_name_1 then modifies var_name_1 and var_name_2 then sends var_name_2 to model 1

For the ocean-atmosphere model we need to send data back from model 1 to model 2 after model 1 has computed. 
This means that I need a step, e.g.

	call gmcfPostModel1(v1sum,v2sum)
	
Furthermore, model 2 will need to receive these data, but obviously the first time there is nothing to receive.

## 2017-06-14

### Model configurations and interactions

#### Atmosphere model: 

        Delta t = 20 min
        
I call this the `GMCF_TIME_QUANTUM_MIN`

#### Ocean model: 

        Delta t = 480 min = 20*24 

#### Coupling 

Coupling period is 10 days (and at t=0 no coupling) = `10*24*60` min or `10*24*3` quanta
i.e. the atmosphere model takes 24 steps for 1 step of the ocean model
Totally ad-hoc, I set the offset to 1 quantum for uvw to ocean and 7 quanta for t to atmosphere

* Ocean is more fine-grained than Atmosphere
=> In ocean I must interpolate the Atmosphere values
=> In atmosphere I must sample the ocean values	
		        
#### Scenario

I will implement the following scenario:

- The atmosphere is much larger than the ocean (twice in every direction) but the resolution is say 5x less
- I will exchange the values between the models at different times. If we couple every 10 days, I assume we start with the atmosphere driving the ocean, and 8 days later the ocean drives the atmosphere.

This means that I need the ability to distinguish between such periodic events. 
In general, we can have k period events. They each have an offset from the start of the simulation and a period which is a multiple of the  time step of the slowest model.
For each of these k events we need to sync, and then take action for the appropriate event. 

For example, suppose the model time step is 20;  we have 3 exchange cycles: 40, 80 and 200 with an offset of resp. 20, 60 and 160
Reducing this it means

    (2,10);(4,6);(10,8)

the corresponding test is 

    (t-t_offset) >0 and (t-t_offset) % exchange_period ==0   
    
#### Generalisation to multiple models

The above is OK for 2 models. For multiple models, what we need is to check for each model if there is a sync point with a given other model. In other words, the tuple should also include the participants:

    (SYNC_PERIOD,SYNC_OFFSET, participating_models)
    
The data structure ofr `participating_models` is definitely related to the `gmcfConnectivityMatrix`, and assuming the number of models is never huge, we could do a linear search. 

For every model, all we need to know is when the sync points are, not actually what to do. So we can create a redundant list of gmcfSyncConfigurations, i.e. we make it a list of lists of tuples.

For every model, we go through one of the sublists by model index. From that we get the `current_sync_config`
    

In [7]:
dt = 20
gmcfSyncConfigurations = [(2,10),(4,6),(10,8)]

def gmcfCheckSync(sim_time,gmcfSyncConfigurations):
    current_sync_config=0;
    for (exchange_period,t_offset) in gmcfSyncConfigurations:
        current_sync_config+=1
        if sim_time >= t_offset and (sim_time-t_offset) % exchange_period == 0:
            print("\tSync for config ",current_sync_config,(exchange_period*dt,t_offset*dt))
            return current_sync_config       
        
for t in range(50):
    print("t :",t*dt)
    sync_case = gmcfCheckSync(t,gmcfSyncConfigurations)
    
    if sync_case == 1:
        print("Action ",sync_case)
    elif sync_case == 2:
        print("Action ",sync_case)
    elif sync_case == 3:
        print("Action ",sync_case)
        

t : 0
t : 20
t : 40
t : 60
t : 80
t : 100
t : 120
	Sync for config  2 (80, 120)
Action  2
t : 140
t : 160
	Sync for config  3 (200, 160)
Action  3
t : 180
t : 200
	Sync for config  1 (40, 200)
Action  1
t : 220
t : 240
	Sync for config  1 (40, 200)
Action  1
t : 260
t : 280
	Sync for config  1 (40, 200)
Action  1
t : 300
t : 320
	Sync for config  1 (40, 200)
Action  1
t : 340
t : 360
	Sync for config  1 (40, 200)
Action  1
t : 380
t : 400
	Sync for config  1 (40, 200)
Action  1
t : 420
t : 440
	Sync for config  1 (40, 200)
Action  1
t : 460
t : 480
	Sync for config  1 (40, 200)
Action  1
t : 500
t : 520
	Sync for config  1 (40, 200)
Action  1
t : 540
t : 560
	Sync for config  1 (40, 200)
Action  1
t : 580
t : 600
	Sync for config  1 (40, 200)
Action  1
t : 620
t : 640
	Sync for config  1 (40, 200)
Action  1
t : 660
t : 680
	Sync for config  1 (40, 200)
Action  1
t : 700
t : 720
	Sync for config  1 (40, 200)
Action  1
t : 740
t : 760
	Sync for config  1 (40, 200)
Action  1
t : 780
t : 8

Now, what we also need is the information about what to do at a given sync step. I think I'll do this via a case statement.



### Request or Push?

The main question to decide is if there is an advantage to the `request` approach.

I think the request approach is necessary when a model does not always need the same information. But because the whole current setup is compile-time decided, I think we can actually skip the requests and simply send the required data after syncing:

    call gmcfSync(sync_cases, sync_case)
    select (sync_case)
        case UVW_TO_OCEAN:
            call gmcfSend(...)
        case T_FROM_ATMOSPHERE:
            call gmcfWait(...)        
            call gmcfReceive(...)        
        

### Atmosphere model actions

1. The `atmosphere` model will send `wind_profile` to the `ocean` model using the `ATMOSPHERE_SYNC_PERIOD` and `ATMOSPHERE_SYNC_OFFSET`
2. The `atmosphere` model will receive `temp` from the `ocean` model using the `OCEAN_SYNC_PERIOD` and `OCEAN_SYNC_OFFSET`

The actions are then: 
- check for sync point
- sync
- send `wind_profile`
- wait for `temp`
- read `temp`


Currently we have following code:

        t_sync = t_atmosphere / t_sync_step ! means atmosphere steps are smaller and more frequent than ocean
        t_inter = mod(t_atmosphere,t_sync_step) ! for interpolation between two steps of atmosphere
        t_atmosphere = t_atmosphere + 1

This code assumes that there are two  models that exchange info with a fixed period. The new model defines a sync point purely in terms of the quantised time step of the model. 

I will assume that the time step of the original code for every model is simply 1, the start time 0. 
So inside the `gmcfSync` subroutine we multiply this with the appropriate time step and add any required offset.

Suppose the original code would not be so well-behaved then we need to do it a bit more complicated:

    delta_t = DELTA_T_SEC ! just to complicate matters
    do t = t_start, t_stop, t_step 
    ! the model will use this relative time `t` for its computations
    ! The absolute time is 
    t_in_min = t*delta_t/60
    t_norm = (t_in_min / GMCF_TIME_QUANTUM) 

For example if `t_start=3` and `t_step=4` then `t = 3,7,11,15,...`
What we want is the number of quanta elapsed since the start of the simulation.
So first we substract the offset `t_start`: `t-t_start = 0,4,8,12,...`
Then we divide by `t_step`: `t_start`: `(t-t_start)/t_step = 0,1,2,3,...`
Let's call this the `n_ticks`:

    n_ticks = (t-t_start)/t_step


Now we need to relate this to the quanta, and we know that the `GMCF_TIME_QUANTUM` is the GCD. The interval in minutes between two ticks is `t_step*delta_t/60` so to get the number of quanta, we have

    quanta_per_tick = ( t_step*delta_t/60 ) / GMCF_TIME_QUANTUM
    
This is `GMCF_TIME_STEP_ATMOSPHERE`, so the input for `gmcfSync` is the `n_ticks`.

Sanity check: for the ocean model, `Delta_t` = 20 minutes or 20*60 seconds; `t_step=1; t_start=0`

So we have:

    n_ticks = t
    GMCF_TIME_STEP_OCEAN = quanta_per_tick = ( 1*20*60/60 ) / 20 = 1


So `n_ticks` is the value that goes into the `gmcfSync` call. In the call we need the number of quanta elapsed since the start of the simulation, this should be `n_ticks * GMCF_TIME_STEP_ATMOSPHERE + GMCF_TIME_OFFSET_ATMOSPHERE`:

    sim_time = n_ticks * GMCF_TIME_STEP_ATMOSPHERE + GMCF_TIME_OFFSET_ATMOSPHERE
    
We then compare `sim_time` to the possible sync configurations for the model.

The call to `gmcfSync` is blocking. I think we should do nothing else in the `gmcfSyncAtmosphere` call.
Instead, the `gmcfSyncAtmosphere` should return the `current_sync_config` and the `gmcfPreAtmosphere` call should use this to decide if it has to send or receive.

We can make `current_sync_config` and `is_sync_point` module globals, so that we don't have to pass them explicitly between the calls.


### Need for Pre/Post?

The order of events is:
- sync if required
- if sync, exchange data (PRE)
- compute
- if sync, exchange data (POST)

Arguably, we should receive in PRE and send in POST, because that means we compute using newly received values and send newly computed values. 

However, if I do that with two models, then in the PRE stage both models wait for data, so there will be deadlock! To avoid that I should let each model send the data right away, in PRE.
Clearly, both models will then also receive in PRE. After computation they could in principle send the data too, and then that data would be pending in the queue until a sync, and then there would be no need for a send in PRE. But that is unsafe ans serves no particular purpose. So I guess POST has no use.



### Note about organisation of the configurations

At the moment I use parameters such as `GMCF_TIME_STEP_<model>`. This is unpractical. A better way is to create a record type for all the configuration data, and then create an array of records for all models. That will make the overall code much more generic. But what to put in a configuration is not so easy to determine.



### Domain interpolation

I want the atmosphere domain to be twice as large as the ocean domain but with `1/5` of the resolution
So if the ocean is `100 x 100`, the atmosphere will be `200/5 x 200/5`



Now suppose I want the ocean to cover from `30/5` to `130/5` and from `40/5` to `140/5`

    integer, parameter :: ATMOSPHERE_IP=48,ATMOSPHERE_JP=48,ATMOSPHERE_KP=27
            
The ratio is encoded in 

    integer, parameter :: ATMOSPHERE_DI=5,ATMOSPHERE_DJ=5,ATMOSPHERE_DK=1
            
I'm not happy with these names. The correct approach would be to define OCEAN_OFFSET_FROM_ATMOSPHERE_X, OCEAN_OFFSET_FROM_ATMOSPHERE_Y 

    integer, parameter :: ATMOSPHERE_IMS=6,ATMOSPHERE_JMS=8,ATMOSPHERE_KMS=1
    integer, parameter :: ATMOSPHERE_IME=26,ATMOSPHERE_JME=28,ATMOSPHERE_KME=27


### Time interpolation

- Time interpolation actually only works in proper producer-consumer cases
- In a case with mutual exchange of data, interpolation is impossible without introducing an artificial phase shift, so the action of one model will lag on the action of another. With more than two models this would I think quickly result in incorrect simulations.
- The alternative, extrapolation, leads to potential overshoot on maxima and undershoot on minima, as well as introduction of higher-order frequencies in the signal.
- So piecewise constant is the best possible alternative? Maybe not. What we can do is interpolate between the old and the new value, leaving the new value as the last one in the interval.
This requires us to know how many ticks there are between updates. 





### Ocean model actions

1. The `ocean` model will send `temp` to the `atmosphere` model using the `OCEAN_SYNC_PERIOD` and `OCEAN_SYNC_OFFSET`
2. The `ocean` model will receive `wind_profile` from the `atmosphere` model using the `ATMOSPHERE_SYNC_PERIOD` and `ATMOSPHERE_SYNC_OFFSET`

The actions are then: 
- check for sync point
- sync
- send `temp`
- wait for `wind_profile`
- read `wind_profile`

## 2017-06-15

Space interpolation: fortunately we can take this straight from Numerical Recipes and I already wrote most of the code. 
Main change this involves is that I need to provide auxiliary arrays with the latitudinal and longitudinal coordinates, or for the simple case at least the offset and grid size. 


## 2017-06-16

I thought about a test scenario and realised that basically there are two possibilities: 

- Concurrent: the two models sync, exchange data, work, repeat;
- Dependent: the two models sync, model1 sends to model2, model2 works, model2 sends to model1, model1 works, repeat

The concurrent case is I think most representative for e.g. ocean-atmosphere interactions as clearly they both "run" in parallel.


In [2]:
# Concurrent case: 

def f1(v1,v2):
    return v1+v2/2.0

def f2(v1,v2):
    return v1-2*v2
    
v1=2
v2=2
for t in range(0,10):
    v1r = v1
    v2r = v2
    v1 = f1(v1r, v2r)
    v2 = f2(v2r, v1r)    
    print(v1,v2)
    
    

3.0 -2
2.0 -8.0
-2.0 -12.0
-8.0 -8.0
-12.0 8.0
-8.0 32.0
8.0 48.0
32.0 32.0
48.0 -32.0
32.0 -128.0


In [15]:
# Dependent case : Fibonacci

def f1(v):
    return v[0]+v[1]

def f2(v1,v2):
    return [v1[1],v2]
    
v1=0
v2=[1,1]
for t in range(0,10):
    v2r = v2
    v1 = f1(v2r)
    v1r = v1    
    v2 = f2(v2, v1r)    
    print(v2)

[1, 2]
[2, 3]
[3, 5]
[5, 8]
[8, 13]
[13, 21]
[21, 34]
[34, 55]
[55, 89]
[89, 144]


The main task to do to finish step 1 of my action plan is to create the `lat` and `lon` coordinate arrays.

## 2017-06-19

### In terms of the action plan, I think I have now do steps 1, 2 and 3. So now I need to test the system in its current state.

* Test functionality without any interpolation: use the test example above. But what I think I'll do is read the v1 and v2 from the temperature and wind profile arrays. Also, I will make the u,v,w and temp arrays the same size in both models.

!!! FIXME: the `.tdc64` file *must* currently be named `gmfc.tdc64` because it is hardcoded in `gmcfCoupler.cc`

But apart from that, after some fumbling with Fortran print formats

        print 7188, model_id, v1,t_surface(1,1),u(1,1,1)
        7188 format("FORTRAN MODEL ",i1, " v1 = ",f8.1,' = (',f8.1,' + ',f8.1,' / 2 )')
                 
This produces the same output as the example above:

        [ocean-atmosphere-model-coupling]$ ./gmcfCoupler | grep 'v1'
        FORTRAN MODEL 1 v1 =      3.0 = (     2.0 +      2.0 / 2 )
        FORTRAN MODEL 1 v1 =      2.0 = (     3.0 +     -2.0 / 2 )
        FORTRAN MODEL 1 v1 =     -2.0 = (     2.0 +     -8.0 / 2 )
        FORTRAN MODEL 1 v1 =     -8.0 = (    -2.0 +    -12.0 / 2 )
        FORTRAN MODEL 1 v1 =    -12.0 = (    -8.0 +     -8.0 / 2 )
        FORTRAN MODEL 1 v1 =     -8.0 = (   -12.0 +      8.0 / 2 )
        FORTRAN MODEL 1 v1 =      8.0 = (    -8.0 +     32.0 / 2 )
        FORTRAN MODEL 1 v1 =     32.0 = (     8.0 +     48.0 / 2 )
        FORTRAN MODEL 1 v1 =     48.0 = (    32.0 +     32.0 / 2 )
        FORTRAN MODEL 1 v1 =     32.0 = (    48.0 +    -32.0 / 2 )
        FORTRAN MODEL 1 v1 =    -32.0 = (    32.0 +   -128.0 / 2 )


## 2017-06-20

* The next step I think is to use different domain sizes but simply pick a subdomain with the size of the smallest domain for transfers. We can test this by using values from the lower-left and upper-right corners and compute the same function for both of them.

I want this subdomain to be static, and it would be part of the scenario eventually. So I guess we just need a subroutine that takes the original domain array, the target subdomain array and the corner index 4-tuple,

```fortran
    subroutine gmcfExtract2DSubdomain(subd_indices, domain, subdomain)
        integer, dimension(4), intent(In) :: subd_indices
        real(kind=4), dimension(:,:) , intent(In) :: domain
        real(kind=4), dimension(:,:) , intent(InOut) :: subdomain
        subdomain = domain(subd_indices(1):subd_indices(2),subd_indices(3):subd_indices(4))
    end subroutine gmcfExtract2DSubdomain
```    

As the ocean model sends a subdomain to the atmosphere model, we need another subroutine to insert this subdomain into the domain as well:
```fortran    
    subroutine gmcfInsert2DSubdomain(subd_indices, domain, subdomain)
        integer, dimension(4), intent(In) :: subd_indices
        real(kind=4), dimension(:,:) , intent(InOut) :: domain
        real(kind=4), dimension(:,:) , intent(In) :: subdomain
        domain(subd_indices(1):subd_indices(2),subd_indices(3):subd_indices(4))=subdomain
    end subroutine gmcfInsert2DSubdomain
```    

Next step: different time steps, e.g. the ones from original models.

    

In [48]:
# Concurrent case with differnt time steps and a long sync period

quantum = 20 # minutes
sync_period = 1*24*3 # quanta

ocean_sync_period = sync_period # quanta
ocean_timestep = 1 # quantum
ocean_offset = 0 
atmosphere_timestep = 24 # quanta
atmosphere_offset = 0
atmosphere_sync_period = sync_period

def ocean(v1,v2):
    return v1+v2/200.0

def atmosphere(v1,v2):
    return v1-2*v2
    
v1=2
v1b=2
v2=2
v2b=2
n_ticks_ocean=0
n_ticks_atmosphere=0
for t in range(0,770):

    sim_time_ocean=n_ticks_ocean*ocean_timestep+ocean_offset
    sim_time_atmosphere=n_ticks_atmosphere*atmosphere_timestep+atmosphere_offset
#     print(t,sim_time_ocean,sim_time_atmosphere,sim_time_ocean% sync_period==0,sim_time_atmosphere % sync_period==0,sync_period)
# ocean and atmosphere sync 
    if (sim_time_ocean% sync_period==0 and sim_time_atmosphere % sync_period==0):
#         print(t,v1,v2)
#         print('FORTRAN MODEL 1 v1b = %8.1f + %8.1f /200.0' % (v1,v2))
        v1r = v1
        v2r = v2
        
# ocean computes for every t 
    if (t % ocean_timestep == 0):
        n_ticks_ocean+=1
        v1b = ocean(v1, v2r)
        if (t%72==0):
            print('\tFORTRAN MODEL 1 v1b =\t%8.1f = ( %8.1f + %8.1f / 200 )'.expandtabs(4) % (v1b,v1,v2))
        v1=v1b
        
# atmosphere computes 
    if t % atmosphere_timestep == 0:
        n_ticks_atmosphere+=1         
        v2 = atmosphere(v2, v1r)    

    

    FORTRAN MODEL 1 v1b =        2.0 = (      2.0 +      2.0 / 200 )
    FORTRAN MODEL 1 v1b =        2.7 = (      2.7 +    -10.0 / 200 )
    FORTRAN MODEL 1 v1b =       -1.0 = (     -0.9 +    -26.3 / 200 )
    FORTRAN MODEL 1 v1b =      -10.5 = (    -10.4 +    -21.0 / 200 )
    FORTRAN MODEL 1 v1b =      -17.7 = (    -17.9 +     41.1 / 200 )
    FORTRAN MODEL 1 v1b =       -2.4 = (     -3.1 +    148.7 / 200 )
    FORTRAN MODEL 1 v1b =       51.2 = (     50.4 +    167.5 / 200 )
    FORTRAN MODEL 1 v1b =      110.0 = (    110.7 +   -134.8 / 200 )
    FORTRAN MODEL 1 v1b =       58.2 = (     62.1 +   -798.9 / 200 )
    FORTRAN MODEL 1 v1b =     -231.3 = (   -225.5 +  -1171.8 / 200 )
    FORTRAN MODEL 1 v1b =     -646.4 = (   -647.3 +    181.0 / 200 )


The Fortran coupled model yields:

    FORTRAN MODEL 1 v1b =      2.0 = (     2.0 +      2.0 / 200 )
    FORTRAN MODEL 1 v1b =      2.7 = (     2.7 +    -10.0 / 200 )
    FORTRAN MODEL 1 v1b =     -1.0 = (    -0.9 +    -26.3 / 200 )
    FORTRAN MODEL 1 v1b =    -10.5 = (   -10.4 +    -21.0 / 200 )
    FORTRAN MODEL 1 v1b =    -17.7 = (   -17.9 +     41.1 / 200 )
    FORTRAN MODEL 1 v1b =     -2.4 = (    -3.1 +    148.7 / 200 )
    FORTRAN MODEL 1 v1b =     51.2 = (    50.4 +    167.5 / 200 )
    FORTRAN MODEL 1 v1b =    110.0 = (   110.7 +   -134.8 / 200 )
    FORTRAN MODEL 1 v1b =     58.2 = (    62.1 +   -798.9 / 200 )
    FORTRAN MODEL 1 v1b =   -231.3 = (  -225.5 +  -1171.8 / 200 )
    FORTRAN MODEL 1 v1b =   -646.4 = (  -647.3 +    181.0 / 200 )
    
So all is well    